# SQL, Database operations, and analysis
In this notebook we will explore SQL and database operations like 'join'. Then we will use the datasets as input to a simple analysis.

## Key Takeaway
**Spark dataframes are different** 

## In this Noteboook
1. create a context
2. read in data
3. register table with SQL
4. perform SQL queries
5. perform database operations
6. do some analysis

In [6]:

# Initialize the spark environment (takes ~ 1min)
import pyspark
conf = pyspark.SparkConf().setAppName('odl').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
sqlc = pyspark.sql.SQLContext(sc)


In [7]:
import pyspark.sql.functions as sf

## Most important part of today's class

here's where the documentation is: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#

## Read in Data

In [3]:
dataPaths = ["bryce.csv","javy.csv","andre.csv"]

In [2]:
dfbryce = sqlc.read.format("csv").option("header","true").option("inferSchema", "true").load(dataPaths[0])
dfjavyb = sqlc.read.format("csv").option("header","true").option("inferSchema", "true").load(dataPaths[1])
dfandre = sqlc.read.format("csv").option("header","true").option("inferSchema", "true").load(dataPaths[2])

NameError: name 'sqlc' is not defined

In [10]:
dfbryce.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tm: string (nullable = true)
 |-- Lg: string (nullable = true)
 |-- G: integer (nullable = true)
 |-- PA: integer (nullable = true)
 |-- AB: integer (nullable = true)
 |-- R: integer (nullable = true)
 |-- H: integer (nullable = true)
 |-- 2B: integer (nullable = true)
 |-- 3B: integer (nullable = true)
 |-- HR: integer (nullable = true)
 |-- RBI: integer (nullable = true)
 |-- SB: integer (nullable = true)
 |-- CS: integer (nullable = true)
 |-- BB: integer (nullable = true)
 |-- SO: integer (nullable = true)
 |-- BA: double (nullable = true)
 |-- OBP: double (nullable = true)
 |-- SLG: double (nullable = true)
 |-- OPS: double (nullable = true)
 |-- OPS+: integer (nullable = true)
 |-- TB: integer (nullable = true)
 |-- GDP: integer (nullable = true)
 |-- HBP: integer (nullable = true)
 |-- SH: integer (nullable = true)
 |-- SF: integer (nullable = true)
 |-- IBB: integer (nullable = true)
 |-- Pos: st

## Use some SQL
We are going to bring in some SQL for this example

## Register Tables

In [11]:
dfbryce.registerTempTable("bryce")
dfjavyb.registerTempTable("javyb")
dfandre.registerTempTable("andre")

## SQL queries to produce dataframes

In [12]:
Bhits = sqlc.sql(""" SELECT Year,H,AB FROM bryce """)

## Spark Dataframes

Spark allows two distinct kinds of operations by the user.

### Transformations
Transformations are operations that will not be completed at the time you write and execute the code in a cell - they will only get executed once you have called a action. An example of a transformation might be to convert an integer into a float or to filter a set of values.

### Actions
Actions are commands that are computed by Spark right at the time of their execution. They consist of running all of the previous transformations in order to get back an actual result. An action is composed of one or more jobs which consists of tasks that will be executed by the workers in parallel where possible


![](trans_and_actions.png)

In [13]:
Bhits.show()

+----+---+---+
|Year|  H| AB|
+----+---+---+
|2012|144|533|
|2013|116|424|
|2014| 96|352|
|2015|172|521|
|2016|123|506|
|2017|134|420|
|2018|137|550|
+----+---+---+



In [14]:
Bhits.explain(True)

== Parsed Logical Plan ==
'Project ['Year, 'H, 'AB]
+- 'UnresolvedRelation `bryce`

== Analyzed Logical Plan ==
Year: int, H: int, AB: int
Project [Year#12, H#20, AB#18]
+- SubqueryAlias bryce
   +- Relation[Year#12,Age#13,Tm#14,Lg#15,G#16,PA#17,AB#18,R#19,H#20,2B#21,3B#22,HR#23,RBI#24,SB#25,CS#26,BB#27,SO#28,BA#29,OBP#30,SLG#31,OPS#32,OPS+#33,TB#34,GDP#35,... 6 more fields] csv

== Optimized Logical Plan ==
Project [Year#12, H#20, AB#18]
+- Relation[Year#12,Age#13,Tm#14,Lg#15,G#16,PA#17,AB#18,R#19,H#20,2B#21,3B#22,HR#23,RBI#24,SB#25,CS#26,BB#27,SO#28,BA#29,OBP#30,SLG#31,OPS#32,OPS+#33,TB#34,GDP#35,... 6 more fields] csv

== Physical Plan ==
*Project [Year#12, H#20, AB#18]
+- *FileScan csv [Year#12,AB#18,H#20] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/ec2-user/SageMaker/Spark19SpDS6003-001/bbref/bryce.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Year:int,AB:int,H:int>


![](query-plan-generation.png)

In [15]:
sqlc.sql(""" SELECT Year,H FROM bryce WHERE YEAR = 2015 """).show()
sqlc.sql(""" SELECT Year,H FROM bryce WHERE YEAR = 2015 """).explain(True)

+----+---+
|Year|  H|
+----+---+
|2015|172|
+----+---+

== Parsed Logical Plan ==
'Project ['Year, 'H]
+- 'Filter ('YEAR = 2015)
   +- 'UnresolvedRelation `bryce`

== Analyzed Logical Plan ==
Year: int, H: int
Project [Year#12, H#20]
+- Filter (YEAR#12 = 2015)
   +- SubqueryAlias bryce
      +- Relation[Year#12,Age#13,Tm#14,Lg#15,G#16,PA#17,AB#18,R#19,H#20,2B#21,3B#22,HR#23,RBI#24,SB#25,CS#26,BB#27,SO#28,BA#29,OBP#30,SLG#31,OPS#32,OPS+#33,TB#34,GDP#35,... 6 more fields] csv

== Optimized Logical Plan ==
Project [Year#12, H#20]
+- Filter (isnotnull(YEAR#12) && (YEAR#12 = 2015))
   +- Relation[Year#12,Age#13,Tm#14,Lg#15,G#16,PA#17,AB#18,R#19,H#20,2B#21,3B#22,HR#23,RBI#24,SB#25,CS#26,BB#27,SO#28,BA#29,OBP#30,SLG#31,OPS#32,OPS+#33,TB#34,GDP#35,... 6 more fields] csv

== Physical Plan ==
*Project [Year#12, H#20]
+- *Filter (isnotnull(YEAR#12) && (YEAR#12 = 2015))
   +- *FileScan csv [Year#12,H#20] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/ec2-user/SageMaker/Spark19

In [16]:
Jhits = sqlc.sql(""" SELECT Year,H,AB FROM javyb """)
Jhits.show()

+----+---+---+
|Year|  H| AB|
+----+---+---+
|2014| 36|213|
|2015| 22| 76|
|2016|115|421|
|2017|128|469|
|2018|176|606|
+----+---+---+



In [17]:
Ahits = sqlc.sql(""" SELECT Year,H,AB FROM ANDRE """)
Ahits.show()

+----+---+---+
|Year|  H| AB|
+----+---+---+
|1976| 20| 85|
|1977|148|525|
|1978|154|609|
|1979|176|639|
|1980|178|577|
|1981|119|394|
|1982|183|608|
|1983|189|633|
|1984|132|533|
|1985|135|529|
|1986|141|496|
|1987|178|621|
|1988|179|591|
|1989|105|416|
|1990|164|529|
|1991|153|563|
|1992|150|542|
|1993|126|461|
|1994| 70|292|
|1995| 58|226|
+----+---+---+
only showing top 20 rows



## Database Operations: joins

In [18]:
df = Bhits.join(Jhits,Bhits.Year==Jhits.Year).join(Ahits,Bhits.Year==Ahits.Year,'outer')
print(df)
df.explain(True)

DataFrame[Year: int, H: int, AB: int, Year: int, H: int, AB: int, Year: int, H: int, AB: int]
== Parsed Logical Plan ==
Join FullOuter, (Year#12 = Year#158)
:- Join Inner, (Year#12 = Year#85)
:  :- Project [Year#12, H#20, AB#18]
:  :  +- SubqueryAlias bryce
:  :     +- Relation[Year#12,Age#13,Tm#14,Lg#15,G#16,PA#17,AB#18,R#19,H#20,2B#21,3B#22,HR#23,RBI#24,SB#25,CS#26,BB#27,SO#28,BA#29,OBP#30,SLG#31,OPS#32,OPS+#33,TB#34,GDP#35,... 6 more fields] csv
:  +- Project [Year#85, H#93, AB#91]
:     +- SubqueryAlias javyb
:        +- Relation[Year#85,Age#86,Tm#87,Lg#88,G#89,PA#90,AB#91,R#92,H#93,2B#94,3B#95,HR#96,RBI#97,SB#98,CS#99,BB#100,SO#101,BA#102,OBP#103,SLG#104,OPS#105,OPS+#106,TB#107,GDP#108,... 6 more fields] csv
+- Project [Year#158, H#166, AB#164]
   +- SubqueryAlias andre
      +- Relation[Year#158,Age#159,Tm#160,Lg#161,G#162,PA#163,AB#164,R#165,H#166,2B#167,3B#168,HR#169,RBI#170,SB#171,CS#172,BB#173,SO#174,BA#175,OBP#176,SLG#177,OPS#178,OPS+#179,TB#180,GDP#181,... 6 more fields] cs

In [19]:
df.show()

+----+----+----+----+----+----+----+----+----+
|Year|   H|  AB|Year|   H|  AB|Year|   H|  AB|
+----+----+----+----+----+----+----+----+----+
|null|null|null|null|null|null|1990| 164| 529|
|null|null|null|null|null|null|1977| 148| 525|
|2018| 137| 550|2018| 176| 606|null|null|null|
|2015| 172| 521|2015|  22|  76|null|null|null|
|null|null|null|null|null|null|1978| 154| 609|
|null|null|null|null|null|null|1988| 179| 591|
|null|null|null|null|null|null|1994|  70| 292|
|2014|  96| 352|2014|  36| 213|null|null|null|
|null|null|null|null|null|null|1979| 176| 639|
|null|null|null|null|null|null|1991| 153| 563|
|null|null|null|null|null|null|1982| 183| 608|
|null|null|null|null|null|null|1989| 105| 416|
|null|null|null|null|null|null|1996|  16|  58|
|null|null|null|null|null|null|1985| 135| 529|
|null|null|null|null|null|null|1987| 178| 621|
|2016| 123| 506|2016| 115| 421|null|null|null|
|null|null|null|null|null|null|1995|  58| 226|
|null|null|null|null|null|null|1980| 178| 577|
|null|null|nu

In [20]:
df = Bhits.join(Jhits,Bhits.Year==Jhits.Year)
print(df)
df.show()

DataFrame[Year: int, H: int, AB: int, Year: int, H: int, AB: int]
+----+---+---+----+---+---+
|Year|  H| AB|Year|  H| AB|
+----+---+---+----+---+---+
|2014| 96|352|2014| 36|213|
|2015|172|521|2015| 22| 76|
|2016|123|506|2016|115|421|
|2017|134|420|2017|128|469|
|2018|137|550|2018|176|606|
+----+---+---+----+---+---+



## Analysis: aggregate, calculate

In [21]:
CH = Bhits.groupBy().agg(sf.sum('H').alias('CH'))
CAB = Bhits.groupBy().agg(sf.sum('AB').alias('CAB'))

In [22]:
CH.explain(True)

== Parsed Logical Plan ==
'Aggregate [sum('H) AS CH#384]
+- Project [Year#12, H#20, AB#18]
   +- SubqueryAlias bryce
      +- Relation[Year#12,Age#13,Tm#14,Lg#15,G#16,PA#17,AB#18,R#19,H#20,2B#21,3B#22,HR#23,RBI#24,SB#25,CS#26,BB#27,SO#28,BA#29,OBP#30,SLG#31,OPS#32,OPS+#33,TB#34,GDP#35,... 6 more fields] csv

== Analyzed Logical Plan ==
CH: bigint
Aggregate [sum(cast(H#20 as bigint)) AS CH#384L]
+- Project [Year#12, H#20, AB#18]
   +- SubqueryAlias bryce
      +- Relation[Year#12,Age#13,Tm#14,Lg#15,G#16,PA#17,AB#18,R#19,H#20,2B#21,3B#22,HR#23,RBI#24,SB#25,CS#26,BB#27,SO#28,BA#29,OBP#30,SLG#31,OPS#32,OPS+#33,TB#34,GDP#35,... 6 more fields] csv

== Optimized Logical Plan ==
Aggregate [sum(cast(H#20 as bigint)) AS CH#384L]
+- Project [H#20]
   +- Relation[Year#12,Age#13,Tm#14,Lg#15,G#16,PA#17,AB#18,R#19,H#20,2B#21,3B#22,HR#23,RBI#24,SB#25,CS#26,BB#27,SO#28,BA#29,OBP#30,SLG#31,OPS#32,OPS+#33,TB#34,GDP#35,... 6 more fields] csv

== Physical Plan ==
*HashAggregate(keys=[], functions=[sum(cast

In [23]:
CH.show()
CAB.show()

+---+
| CH|
+---+
|922|
+---+

+----+
| CAB|
+----+
|3306|
+----+



In [24]:
rawAVG = CH.take(1)[0][0]/CAB.take(1)[0][0]
print(rawAVG)

0.278886872353297


In [25]:
round(rawAVG,3)

0.279

# Bryce Harper career batting average: 0.279